In [1]:
from typing import List, Dict, Any
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [2]:
# Configuración de autenticación.
SPOTIPY_CLIENT_ID = "bad5756382e744b189437e7d92e1bd1f"
SPOTIPY_CLIENT_SECRET = "9bd792ef5f59442b881154256825b41d"
SPOTIPY_REDIRECT_URI = "http://localhost:8888/callback"

In [8]:
# Configuración de permisos para la biblioteca y playlists.
SCOPE = "user-library-modify user-library-read"

In [9]:
# Autenticación con Spotipy.
Spotify_Client = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
    scope=SCOPE
))

In [10]:
def Remove_All_Albums_From_Library(Spotify_Client: Any) -> None:

    """
    Removes all albums from the user's Spotify library.

    Parameters:
        Spotify_Client (Any): An authenticated Spotify client instance.

    Example:
        >>> from spotipy import Spotify
        >>> Remove_All_Albums_From_Library(Spotify_Client)
        
    """

    # Fetch and delete albums in batches of up to 50.
    Results = Spotify_Client.current_user_saved_albums(limit=50)
    while Results["items"]:
        Album_Ids = [Album["album"]["id"] for Album in Results["items"]]

        # Remove albums in the current batch.
        Spotify_Client.current_user_saved_albums_delete(Album_Ids)
        print(f"Removed {len(Album_Ids)} albums.")

        # Fetch the next batch of albums.
        Results = Spotify_Client.current_user_saved_albums(limit=50)

    print("All albums have been removed from your library.")

In [41]:
def Get_Saved_Albums() -> List[Dict[str, Dict]]:

    """
    Retrieves all saved albums from the user's Spotify library.

    Returns:
        A list of dictionaries containing album details.

    Example:
        >>> Get_Saved_Albums()
        [{'name': 'Album 1', 'uri': 'spotify:album:123', 'tracks': {...}}, ...]

    """

    Saved_Albums = []
    Results = Spotify_Client.current_user_saved_albums(limit=50)

    while Results:
        for Item in Results['items']:
            Album = Item['album']
            Saved_Albums.append(Album)

        # Check if there are more albums to paginate through.
        Results = Spotify_Client.next(Results) if Results['next'] else None

    return Saved_Albums

In [42]:
def Create_Playlists_From_Albums(Albums: List[Dict[str, Dict]]) -> None:

    """
    Creates a playlist for each album and adds all its tracks.

    Parameters:
        Albums: A list of album dictionaries from the Spotify API.

    Returns:
        None.

    Example:
        >>> Albums = Get_Saved_Albums()
        >>> Create_Playlists_From_Albums(Albums)

    """

    User_Id = Spotify_Client.current_user()['id']

    for Album in Albums:
        Album_Name = Album['name'].capitalize()  # Format album name.
        Artist_Name = Album['artists'][0]['name']  # Get the first artist's name.
        Release_Year = Album['release_date'][:4]  # Extract the year from release date.

        Playlist_Name = f"{Artist_Name}. {Album_Name} ({Release_Year})"

        Album_Tracks = [Track['uri'] for Track in Album['tracks']['items']]

        # Create a new playlist for the album.
        Playlist = Spotify_Client.user_playlist_create(
            user=User_Id,
            name=Playlist_Name,
            public=False
        )
        Playlist_Id = Playlist['id']

        # Add tracks to the created playlist.
        Spotify_Client.user_playlist_add_tracks(
            user=User_Id,
            playlist_id=Playlist_Id,
            tracks=Album_Tracks
        )
        print(f"Playlist created: {Playlist_Name}")

In [43]:
def Delete_All_Playlists_Starting_With(
    Prefix: str,
    Client_ID: str,
    Client_Secret: str,
    Redirect_URI: str
) -> None:

    """
    Deletes all playlists whose names start with a specified prefix.

    Parameters:
        Prefix: A string representing the prefix to match playlist names.
        Client_ID: Spotify API Client ID.
        Client_Secret: Spotify API Client Secret.
        Redirect_URI: Redirect URI for Spotify authentication.

    Returns:
        None.

    Example:
        >>> Delete_All_Playlists_Starting_With("Album:", "your_client_id", 
        >>>                                     "your_client_secret", 
        >>>                                     "http://localhost:8888/callback")

    """

    from spotipy import Spotify
    from spotipy.oauth2 import SpotifyOAuth

    # Authenticate using the provided credentials.
    Spotify_Client = Spotify(auth_manager=SpotifyOAuth(
        client_id=Client_ID,
        client_secret=Client_Secret,
        redirect_uri=Redirect_URI,
        scope="playlist-read-private playlist-modify-private playlist-modify-public"
    ))

    User_Id = Spotify_Client.current_user()['id']  # Fetch the current user's ID.
    Playlists = Spotify_Client.current_user_playlists(limit=50)

    while Playlists:
        for Playlist in Playlists['items']:
            if Playlist['name'].startswith(Prefix):
                # Unfollow (delete) the playlist.
                Spotify_Client.user_playlist_unfollow(user=User_Id, playlist_id=Playlist['id'])
                print(f"Playlist deleted: {Playlist['name']}")

        # Fetch the next batch of playlists if available.
        Playlists = Spotify_Client.next(Playlists) if Playlists['next'] else None

In [ ]:
# Crear playlist de álbumes.
if __name__ == "__main__":

    """
    Main execution script to fetch saved albums and create playlists.
    
    """

    #print("Fetching saved albums...")
    #Saved_Albums = Get_Saved_Albums()
    #print(f"Found {len(Saved_Albums)} albums.")

    #Create_Playlists_From_Albums(Saved_Albums)

In [11]:
# Borrar todos los álbumes de la biblioteca.
Remove_All_Albums_From_Library(Spotify_Client)

Removed 50 albums.
Removed 50 albums.
Removed 50 albums.
Removed 50 albums.
Removed 50 albums.
Removed 50 albums.
Removed 50 albums.
Removed 50 albums.
Removed 50 albums.
Removed 3 albums.
All albums have been removed from your library.
